In [1]:
import sys
import os

# Add the build directory to Python path
build_dir = os.path.abspath('/Users/kisel/uni/apal/Algorithm-Selector/Implementation/lib')
sys.path.insert(0, build_dir)

# Add the data directory to Python path
data_dir = os.path.abspath('/Users/kisel/uni/apal/Algorithm-Selector/Implementation/data')
sys.path.insert(0, data_dir)

import pandas as pd
import numpy as np
from tqdm import tqdm

import sorters
import probes
import time

import list_generators as lg
import runtime_features as rf

In [2]:
data = pd.read_feather("/Users/kisel/uni/apal/Algorithm-Selector/Implementation/data/varied_dataset.feather")
data = data.head(10000)
data['data'] = data['data'].apply(lambda arr: arr.tolist())
data

,list_type,data
0,random,"[3.0, 4.0, 5.0, 1.0, 0.0, 7.0, 8.0, 9.0, 2.0, ..."
1,random,"[9.0, 4.0, 7.0, 8.0, 0.0, 1.0, 2.0, 3.0, 5.0, ..."
2,random,"[7.0, 6.0, 9.0, 2.0, 8.0, 1.0, 5.0, 3.0, 0.0, ..."
3,random,"[6.0, 8.0, 7.0, 4.0, 2.0, 0.0, 3.0, 9.0, 5.0, ..."
4,random,"[3.0, 2.0, 4.0, 9.0, 6.0, 5.0, 7.0, 0.0, 8.0, ..."
...,...,...
695,sinewave,"[-0.02756555710217877, 0.797170655956496, 0.92..."
696,sinewave,"[0.08669128387501127, 0.7731831498455023, 0.83..."
697,sinewave,"[-0.06389230113073735, 0.9096311765934534, 0.8..."
698,sinewave,"[-0.056485825525354555, 0.8336981801258728, 0...."


In [3]:
tqdm.pandas()
data['features'] = data['data'].progress_apply(rf.extract_features)
data

100%|██████████| 700/700 [00:05<00:00, 130.29it/s] 


,list_type,data,features
0,random,"[3.0, 4.0, 5.0, 1.0, 0.0, 7.0, 8.0, 9.0, 2.0, ...","{'size': 10, 'range': 9.0, 'avg_dup_pos': 0.0,..."
1,random,"[9.0, 4.0, 7.0, 8.0, 0.0, 1.0, 2.0, 3.0, 5.0, ...","{'size': 10, 'range': 9.0, 'avg_dup_pos': 0.0,..."
2,random,"[7.0, 6.0, 9.0, 2.0, 8.0, 1.0, 5.0, 3.0, 0.0, ...","{'size': 10, 'range': 9.0, 'avg_dup_pos': 0.0,..."
3,random,"[6.0, 8.0, 7.0, 4.0, 2.0, 0.0, 3.0, 9.0, 5.0, ...","{'size': 10, 'range': 9.0, 'avg_dup_pos': 0.0,..."
4,random,"[3.0, 2.0, 4.0, 9.0, 6.0, 5.0, 7.0, 0.0, 8.0, ...","{'size': 10, 'range': 9.0, 'avg_dup_pos': 0.0,..."
...,...,...,...
695,sinewave,"[-0.02756555710217877, 0.797170655956496, 0.92...","{'size': 100000, 'range': 2.199329377496495, '..."
696,sinewave,"[0.08669128387501127, 0.7731831498455023, 0.83...","{'size': 100000, 'range': 2.199580949990282, '..."
697,sinewave,"[-0.06389230113073735, 0.9096311765934534, 0.8...","{'size': 100000, 'range': 2.1998597642384285, ..."
698,sinewave,"[-0.056485825525354555, 0.8336981801258728, 0....","{'size': 100000, 'range': 2.1992526202659786, ..."


In [4]:
from pprint import pprint

for idx, row in data.iterrows():
    print(f"Row {idx}:")
    pprint(row["features"])
    print("-" * 50)

Row 0:
{'avg_dup_distinct': 0.0,
 'avg_dup_pos': 0.0,
 'categorical_kurtosis': 0.0,
 'categorical_skew': 0.0,
 'dis': 8,
 'entropy': 3.321928094887362,
 'mono': 3,
 'range': 9.0,
 'runs': 3,
 'size': 10}
--------------------------------------------------
Row 1:
{'avg_dup_distinct': 0.0,
 'avg_dup_pos': 0.0,
 'categorical_kurtosis': 0.0,
 'categorical_skew': 0.0,
 'dis': 9,
 'entropy': 3.321928094887362,
 'mono': 2,
 'range': 9.0,
 'runs': 2,
 'size': 10}
--------------------------------------------------
Row 2:
{'avg_dup_distinct': 0.0,
 'avg_dup_pos': 0.0,
 'categorical_kurtosis': 0.0,
 'categorical_skew': 0.0,
 'dis': 9,
 'entropy': 3.321928094887362,
 'mono': 4,
 'range': 9.0,
 'runs': 5,
 'size': 10}
--------------------------------------------------
Row 3:
{'avg_dup_distinct': 0.0,
 'avg_dup_pos': 0.0,
 'categorical_kurtosis': 0.0,
 'categorical_skew': 0.0,
 'dis': 9,
 'entropy': 3.321928094887362,
 'mono': 3,
 'range': 9.0,
 'runs': 6,
 'size': 10}
-------------------------------

In [5]:
def find_best_sorter(arr):
    best_name, best_time = None, float("inf")
    for sorter in sorters.list_sorters():
        time = sorters.benchmark_sorter(sorter, arr)
        if time < best_time:
            best_time, best_name = time, sorter

    return best_name

In [6]:
tqdm.pandas()
data['best_sort'] = data['data'].progress_apply(find_best_sorter)

100%|██████████| 700/700 [01:08<00:00, 10.26it/s]  


In [7]:
data

,list_type,data,features,best_sort
0,random,"[3.0, 4.0, 5.0, 1.0, 0.0, 7.0, 8.0, 9.0, 2.0, ...","{'size': 10, 'range': 9.0, 'avg_dup_pos': 0.0,...",merge_sort
1,random,"[9.0, 4.0, 7.0, 8.0, 0.0, 1.0, 2.0, 3.0, 5.0, ...","{'size': 10, 'range': 9.0, 'avg_dup_pos': 0.0,...",quick_merge_sort
2,random,"[7.0, 6.0, 9.0, 2.0, 8.0, 1.0, 5.0, 3.0, 0.0, ...","{'size': 10, 'range': 9.0, 'avg_dup_pos': 0.0,...",insertion_sort
3,random,"[6.0, 8.0, 7.0, 4.0, 2.0, 0.0, 3.0, 9.0, 5.0, ...","{'size': 10, 'range': 9.0, 'avg_dup_pos': 0.0,...",merge_sort
4,random,"[3.0, 2.0, 4.0, 9.0, 6.0, 5.0, 7.0, 0.0, 8.0, ...","{'size': 10, 'range': 9.0, 'avg_dup_pos': 0.0,...",merge_sort
...,...,...,...,...
695,sinewave,"[-0.02756555710217877, 0.797170655956496, 0.92...","{'size': 100000, 'range': 2.199329377496495, '...",std_sort
696,sinewave,"[0.08669128387501127, 0.7731831498455023, 0.83...","{'size': 100000, 'range': 2.199580949990282, '...",std_sort
697,sinewave,"[-0.06389230113073735, 0.9096311765934534, 0.8...","{'size': 100000, 'range': 2.1998597642384285, ...",std_sort
698,sinewave,"[-0.056485825525354555, 0.8336981801258728, 0....","{'size': 100000, 'range': 2.1992526202659786, ...",std_sort


In [8]:
data.to_feather("/Users/kisel/uni/apal/Algorithm-Selector/Implementation/data/data_and_features.feather")